In [1]:
import torch

def mse(img1, img2):
    return (((img1 - img2)) ** 2).view(img1.shape[0], -1).mean(1, keepdim=True)

def psnr(img1, img2):
    mse = (((img1 - img2)) ** 2).view(img1.shape[0], -1).mean(1, keepdim=True)
    return 20 * torch.log10(1.0 / torch.sqrt(mse))


In [75]:
import os
import h5py
import cv2
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
from pathlib import Path
import sys
import yaml
from munch import munchify
from math import atan
from collections import OrderedDict

sys.path.append("D:/gs-localization/gaussian_splatting")
sys.path.append("D:/gs-localization")
sys.path.append("D:/gs-localization/gs_localization/pipelines")


from tools.config_utils import load_config, update_recursive
from tools import read_write_model
from tools.gaussian_model import GaussianModel
from tools import render
from tools.camera_utils import Camera
from tools.descent_utils import get_loss_tracking
from tools.pose_utils import update_pose
from tools.graphics_utils import getProjectionMatrix2


def gradient_decent(viewpoint, config, initial_R, initial_T):

    viewpoint.update_RT(initial_R, initial_T)
    
    opt_params = []
    opt_params.append(
        {
            "params": [viewpoint.cam_rot_delta],
            "lr": 0.000001,
            "name": "rot_{}".format(viewpoint.uid),
        }
    )
    opt_params.append(
        {
            "params": [viewpoint.cam_trans_delta],
            "lr": 0.001,
            "name": "trans_{}".format(viewpoint.uid),
        }
    )
    opt_params.append(
        {
            "params": [viewpoint.exposure_a],
            "lr": 0.001,
            "name": "exposure_a_{}".format(viewpoint.uid),
        }
    )
    opt_params.append(
        {
            "params": [viewpoint.exposure_b],
            "lr": 0.001,
            "name": "exposure_b_{}".format(viewpoint.uid),
        }
    )
    

    pose_optimizer = torch.optim.Adam(opt_params)
    
    for tracking_itr in range(50):
        
        render_pkg = render(
            viewpoint, Model, pipeline_params, background
        )
        
        image, depth, opacity = (
            render_pkg["render"],
            render_pkg["depth"],
            render_pkg["opacity"],
        )
          
        pose_optimizer.zero_grad()
        
        loss_tracking = get_loss_tracking(
            config, image, depth, opacity, viewpoint
        )
        loss_tracking.backward()
        
    
        with torch.no_grad():
            pose_optimizer.step()
            converged = update_pose(viewpoint, converged_threshold=1e-4)
    
        if converged:
            break
             
    return viewpoint.R, viewpoint.T, render_pkg


class Transformation:
    def __init__(self, R=None, T=None):
        self.R = R
        self.T = T

def quat_to_rotmat(qvec):
    qvec = np.array(qvec, dtype=float)
    w, x, y, z = qvec
    R = np.array([
        [1 - 2*y**2 - 2*z**2, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x**2 - 2*y**2]
    ])
    return R


def focal2fov(focal, pixels):
    return 2 * atan(pixels / (2 * focal))

def load_pose(pose_txt):
    pose = []
    with open(pose_txt, 'r') as f:
        for line in f:
            row = line.strip('\n').split()
            row = [float(c) for c in row]
            pose.append(row)
    pose = np.array(pose).astype(np.float32)
    assert pose.shape == (4,4)
    return pose

def create_mask(mkpts_lst, width, height, k):
    # Initial mask as all False
    mask = np.zeros((height, width), dtype=bool)
    
    # Calculat k radius
    half_k = k // 2
    
    # Iterate through all points
    for pt in mkpts_lst:
        x, y = int(pt[0]), int(pt[1])
        
        # Calculate k*k borders
        x_min = max(0, x - half_k)
        x_max = min(width, x + half_k + 1)
        y_min = max(0, y - half_k)
        y_max = min(height, y + half_k + 1)
        
        # Set mask k*k area as True
        mask[y_min:y_max, x_min:x_max] = True
    
    # Shape: (1, height, width)
    mask = mask[np.newaxis, :, :]
    
    return mask

class BaseDataset(torch.utils.data.Dataset):
    def __init__(self, args, path, config):
        self.args = args
        self.path = path
        self.config = config
        self.device = "cuda:0"
        self.dtype = torch.float32
        self.num_imgs = 999

    def __len__(self):
        return self.num_imgs

    def __getitem__(self, idx):
        pass

class MonocularDataset(BaseDataset):
    def __init__(self, args, path, config):
        super().__init__(args, path, config)
        calibration = config["Dataset"]["Calibration"]

        # depth parameters
        self.has_depth = True if "depth_scale" in calibration.keys() else False
        self.depth_scale = calibration["depth_scale"] if self.has_depth else None


    def __getitem__(self, idx):
        color_path = self.color_paths[idx]
        pose = self.poses[idx]
        intrinsic = self.intrinsics[idx]

        image = np.array(Image.open(color_path))
        depth = None

        if self.has_depth:
            depth_path = self.depth_paths[idx]
            depth = np.array(Image.open(depth_path)) / self.depth_scale

        image = (
            torch.from_numpy(image / 255.0)
            .clamp(0.0, 1.0)
            .permute(2, 0, 1)
            .to(device=self.device, dtype=self.dtype)
        )
        pose = torch.from_numpy(pose).to(device=self.device)
        intrinsic = torch.from_numpy(intrinsic).to(device=self.device)
        projection_matrix = self.projection_matrices[idx].to(device="cuda:0")
        
        return image, depth, pose, intrinsic, projection_matrix 

def parse_camera_params(file_path):
    camera_params = {}

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            
            # Ensure there are at least 8 elements, and the second element is 'PINHOLE'
            if len(parts) == 8 and parts[1] == 'PINHOLE':
                try:
                    # Extract parameters, skipping the second item 'PINHOLE'
                    img_name = parts[0]
                    w = int(parts[2])
                    h = int(parts[3])
                    fx = float(parts[4])
                    fy = float(parts[5])
                    cx = float(parts[6])
                    cy = float(parts[7])
                    
                    # Store in the dictionary
                    camera_params[img_name] = {
                        'fx': fx,
                        'fy': fy,
                        'w': w,
                        'h': h,
                        'cx': cx,
                        'cy': cy
                    }
                except ValueError as e:
                    print(f"Error parsing line: {line}. Error: {e}")
            else:
                print(f"Line skipped due to incorrect format: {line}")
    
    return camera_params
    
class cambridge_Dataset(MonocularDataset):
    def __init__(self, args, path, config, data_folder, scene):
        super().__init__(args, path, config)
        self.has_depth = False
        self.cambridge_Parser(data_folder, scene) 
        
    def cambridge_Parser(self, data_folder, scene):
        self.color_paths, self.poses, self.depth_paths, self.intrinsics, self.projection_matrices = [], [], [], [], []

        gt_dirs = Path(data_folder) / f"CambridgeLandmarks_Colmap_Retriangulated_1024px/{scene}/empty_all"
        _, images, _ = read_write_model.read_model(gt_dirs, ".txt")
        output_folder = data_folder.replace("datasets", "output")
        
        # Read the filenames from test_fewshot.txt and store them in a set.
        test_images_path = Path(data_folder) / scene / "test_full.txt"
        
        with open(test_images_path, 'r') as f:
            test_images = set(line.strip() for line in f)

        intrinsics = parse_camera_params(f"D:/gs-localization/output/cambridge_full/{scene}/query_list_with_intrinsics.txt")

        for i, image in tqdm(images.items(),"Load dataset"):
            image_name = image.name.replace("/","_")
            # Execute the following operation only if image.name exists in test_images."
            if image_name in test_images:
                image_path = Path(data_folder) / scene / 'images_full' / image_name
                self.color_paths.append(image_path)
                R_gt, t_gt = image.qvec2rotmat(), image.tvec
                pose = np.eye(4)            
                pose[:3, :3] = R_gt         
                pose[:3, 3] = t_gt 
                self.poses.append(pose)
                self.depth_paths.append(None)
                
                intrinsic = intrinsics[image_name]     
                projection_matrix = getProjectionMatrix2(
                    znear=0.01,
                    zfar=100.0,
                    fx=intrinsic["fx"],
                    fy=intrinsic["fy"],
                    cx=intrinsic["cx"],
                    cy=intrinsic["cy"],
                    W=intrinsic["w"],
                    H=intrinsic["h"],
                ).transpose(0, 1)
    
                self.intrinsics.append(np.array([
                                       intrinsic["fx"],
                                       intrinsic["fy"],
                                       intrinsic["cx"],
                                       intrinsic["cy"],
                                       focal2fov(intrinsic["fx"], intrinsic["w"]),
                                       focal2fov(intrinsic["fy"], intrinsic["h"]),
                                       intrinsic["h"], 
                                       intrinsic["w"]
                                        ]))
                
                self.projection_matrices.append(projection_matrix)


        # Sort self.color_paths, self.poses, and self.depth_paths based on normal file name order
        sorted_data = sorted(zip(self.color_paths, self.depth_paths, self.poses, 
                                 self.intrinsics, self.projection_matrices), key=lambda x: x[0].name)
        self.color_paths, self.depth_paths, self.poses, self.intrinsics, self.projection_matrices = zip(*sorted_data)
        del images

with open("D:/gs-localization/gs_localization/pipelines/configs/mono/tum/fr3_office.yaml", "r") as f:
    cfg_special = yaml.full_load(f)

inherit_from = "D:/gs-localization/gs_localization/pipelines/configs/mono/tum/base_config.yaml"

if inherit_from is not None:
    cfg = load_config(inherit_from)
else:
    cfg = dict()

data_folder = "D:/gs-localization/datasets/cambridge"
config = update_recursive(cfg, cfg_special)
config = cfg
config["Training"]["monocular"] = True
config["Training"]["opacity_threshold"] = 0.99
config["Training"]["edge_threshold"] = 1.1
for scene in ['StMarysChurch']:
    try:
        Model = GaussianModel(3, config)
        Model.load_ply(f"D:/gs-localization/output/cambridge_full/{scene}/gs_map/iteration_30000/point_cloud.ply")
    except:
        Model = GaussianModel(1, config)
        Model.load_ply(f"D:/gs-localization/output/cambridge_full/{scene}/gs_map/iteration_30000/point_cloud.ply")
    
    model_params = munchify(config["model_params"])
    pipeline_params = munchify(config["pipeline_params"])
    data_folder = "D:/gs-localization/datasets/cambridge"
    dataset = cambridge_Dataset(model_params, model_params.source_path, config, data_folder, scene)
    bg_color = [0, 0, 0] 
    background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")
    
    # use OrderedDict to substitute defaultdict
    test_infos = OrderedDict()
    
    # suppose file open and read
    with open(f"D:/gs-localization/output/cambridge_full/{scene}/results_sparse.txt", "r") as f:
        for line in f:
            parts = line.strip().split()
            name = parts[0]
            qvec = list(map(float, parts[1:5]))
            tvec = list(map(float, parts[5:8]))

            R = quat_to_rotmat(qvec)
            T = np.array(tvec)
    
            # insert directly in OrderedDict
            test_infos[name] = Transformation(R=R, T=T)
    
    # sort OrderedDict according to name 
    test_infos = OrderedDict(sorted(test_infos.items(), key=lambda item: item[0]))
    
    rot_errors = []
    trans_errors = []
    
    file = h5py.File(f'D:/gs-localization/output/cambridge_full/{scene}/feats-superpoint-n4096-r1024.h5', 'r')



Load dataset: 100%|█████████████████████████████████████████████████████████████| 2017/2017 [00:00<00:00, 58310.03it/s]


In [76]:
import numpy as np
print(scene)
# 加载 trans_errors.npy 文件
trans_errors_path = f'D:/gs-localization/output/cambridge_full/{scene}/trans_errors.npy'
trans_errors = np.load(trans_errors_path)

# 计算中值
median_value = np.median(trans_errors)

# 找到最接近中值的索引
median_index = np.argmin(np.abs(trans_errors - median_value))
print(median_index)

# 打开 results_sparse.txt 文件并提取对应行的名字
results_path = f'D:/gs-localization/output/cambridge_full/{scene}/results_sparse.txt'
with open(results_path, 'r') as f:
    lines = f.readlines()

# 提取每一行的第一个字段（假设是名字）
names = [line.split()[0] for line in lines]

# 根据找到的索引获取对应的名字
median_name = names[median_index]

# 输出结果
print(f"最接近中值错误的图片名字: {median_name}")


StMarysChurch
21
最接近中值错误的图片名字: seq13_frame00176.png


In [77]:
i = 21
image = 21
viewpoint = Camera.init_from_intrinsic(dataset, i)
viewpoint.compute_grad_mask(config)
viewpoint.R = viewpoint.R_gt
viewpoint.T = viewpoint.T_gt
initial_R = torch.tensor(viewpoint.R_gt)
initial_T = torch.tensor(viewpoint.T_gt).squeeze()
print("???")
gradient_decent(viewpoint, config, initial_R, initial_T)
print("???")

file.close()

render_pkg = render(
            viewpoint, Model, pipeline_params, background
        )




C:\Users\27118\AppData\Local\Temp\ipykernel_30920\907923877.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  initial_R = torch.tensor(viewpoint.R_gt)
C:\Users\27118\AppData\Local\Temp\ipykernel_30920\907923877.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  initial_T = torch.tensor(viewpoint.T_gt).squeeze()


???
???


In [78]:
from PIL import Image, ImageDraw
import torchvision.transforms as transforms
import numpy as np

# 假设 viewpoint.original_image 和 render_pkg["render"] 是 Tensor
ground_truth_tensor = viewpoint.original_image
localized_tensor = render_pkg["render"]

# 找到每个像素中 R、G、B 三个通道的最大值
max_vals, _ = localized_tensor.max(dim=0)  # 得到每个像素的最大值 (H, W)

# 找到哪些像素的最大值超过 1
exceeds_one_mask = max_vals > 1  # 布尔掩码，标记哪些像素的最大值超过 1

# 对超过 1 的地方，将 R、G、B 值同时按最大值进行归一化
localized_tensor[:, exceeds_one_mask] = localized_tensor[:, exceeds_one_mask] / (max_vals[exceeds_one_mask] )

# 将 Tensor 转换为 PIL 图像
tensor_to_pil = transforms.ToPILImage()

ground_truth_image = tensor_to_pil(ground_truth_tensor)
localized_image = tensor_to_pil(localized_tensor)

# 确保两张图片大小相同（可以选择调整大小）
width, height = ground_truth_image.size
localized_image = localized_image.resize((width, height))

# 创建一个新的空白图像，用来合成 ground truth 和 localized image
combined_image = Image.new('RGB', (width, height))

# 将图像转换为 NumPy 数组，方便逐像素操作
ground_truth_array = np.array(ground_truth_image)
localized_image_array = np.array(localized_image)

# 根据条件 x < ay 来合成图像
for y in range(height):
    for x in range(width):
        if x < (y * (width / height)):  # 根据比例 x < ay 来判断
            combined_image.putpixel((x, y), tuple(ground_truth_array[y, x]))  # 放置 ground truth
        else:
            combined_image.putpixel((x, y), tuple(localized_image_array[y, x]))  # 放置 localized image

# 绘制白色虚线对角线
draw = ImageDraw.Draw(combined_image)
line_length = 20  # 每个虚线段的长度
gap_length = 10   # 每段虚线之间的间隔

# 计算对角线的总长度
diagonal_length = int((width**2 + height**2)**0.5)

# 循环绘制虚线
for i in range(0, diagonal_length, line_length + gap_length):
    start_x = int(i * (width / diagonal_length))  # 起点 x
    start_y = int(i * (height / diagonal_length))  # 起点 y
    end_x = int((i + line_length) * (width / diagonal_length))  # 终点 x
    end_y = int((i + line_length) * (height / diagonal_length))  # 终点 y

    # 绘制虚线的段
    draw.line((start_x, start_y, end_x, end_y), fill="white", width=3)

# 画小框
small_box_start = (200, 150)  # 小框左上角起始点 (x, y)
small_box_width = 150         # 小框的宽度
small_box_height = 100        # 小框的高度
small_box_end = (small_box_start[0] + small_box_width, small_box_start[1] + small_box_height)

# 绘制蓝色小框
draw.rectangle([small_box_start, small_box_end], outline="green", width=2)

# 提取小框中的部分
small_box_region = combined_image.crop((small_box_start[0], small_box_start[1], small_box_end[0], small_box_end[1]))

# 放大小框中的部分
scale_factor = 1.6  # 放大倍数
large_box_region = small_box_region.resize((int(small_box_width * scale_factor), int(small_box_height * scale_factor)))

# 将放大的大框放置在小框旁边，覆盖图片部分区域
large_box_start_x = small_box_start[0]   # 小框右边再加10像素
large_box_start_y = small_box_start[1] + small_box_width - 20

# 确保大框不会超出图片边界
if large_box_start_x + large_box_region.width > width:
    large_box_start_x = width - large_box_region.width - 10
if large_box_start_y + large_box_region.height > height:
    large_box_start_y = height - large_box_region.height - 10

# 将放大的区域粘贴回原图中
combined_image.paste(large_box_region, (large_box_start_x, large_box_start_y))

# 画大框
large_box_end = (large_box_start_x + large_box_region.width, large_box_start_y + large_box_region.height)
draw.rectangle([large_box_start_x, large_box_start_y, large_box_end[0], large_box_end[1]], outline="green", width=3)


ground_truth_image.save(f"C:/Users/27118/Desktop/pics2/{scene}_orig.png")
localized_image.save(f"C:/Users/27118/Desktop/pics2/{scene}_loc.png")
localized_image.show()

In [38]:
from PIL import Image

# 场景顺序
scenes = ["chess", "fire", "heads", "office", "pumpkin", "redkitchen", "stairs"]

# 图片路径模板
image_paths = [f"C:/Users/27118/Desktop/{scene}_compare.png" for scene in scenes]

# 打开所有图片，并获取宽高信息
images = [Image.open(img_path) for img_path in image_paths]
width, height = images[0].size  # 假设所有图片大小相同

# 创建一个白色的图片，用作第 8 张图
white_image = Image.new('RGB', (width, height), color='white')

# 添加白色图片到 images 列表中，确保最后总共有 8 张图片
images.append(white_image)

# 创建一个新的空白图像，宽度为4张图片的宽度，高度为两行的图片高度
new_image = Image.new('RGB', (width * 4, height * 2))

# 逐一粘贴图片到新图像中
for i, img in enumerate(images):
    # 计算每张图片的位置
    x_offset = (i % 4) * width  # 每行最多放置4张图片
    y_offset = (i // 4) * height  # 放置到第几行
    new_image.paste(img, (x_offset, y_offset))

# 显示合成后的图像
new_image.show()

# 保存最终合成图像
new_image.save("C:/Users/27118/Desktop/combined_image.png")


In [39]:
import numpy as np

# 指定.npy文件的路径
f"D:/gs-localization/output/7scenes/{scene}/results_sparse.txt"
file_path = 'D:/gs-localization/output/7scenes/{scene}/rot_errors.npy'

# 加载.npy文件
data = np.load(file_path)

# 查看文件内容
print(np.median(data))


FileNotFoundError: [Errno 2] No such file or directory: 'D:/gs-localization/output/7scenes/{scene}/rot_errors.npy'